In [1]:
from datetime import datetime
import os

import dask.dataframe as dd
from dask.distributed import Client
import numpy as np
import pandas as pd
from pyprojroot import here

In [2]:
def extract_matched_sentences_i(sent_set, terms):
    #breakpoint()
    ss = [set(x) for x in sent_set]
    term_idx = {k:[] for k in terms.keys()}
    for idx, st in enumerate(ss):
        for tm in terms.keys():
            #print(idx, st, tm)
            tm_st = set(tm.split(" "))
            if tm_st.issubset(st):
                term_idx[tm].append(idx)
    return term_idx

t1 = [['my', 'name', 'is', 'daniel'], ['your', 'name', 'is', 'eric']]
t2 = {'daniel': True, 'name': True}
extract_matched_sentences_i(t1, t2) == {'daniel': [0], 'name': [0, 1]}

t3 = extract_matched_sentences_i(t1, t2)
t3

def extract_matched_sentences(sentences, term_idx_dict):
    matched_sentences = {k: [] for k in term_idx_dict.keys()}
    for tm, idx in term_idx_dict.items():
        matched_sentences[tm].append(
            [sentences[i] for i in idx]
        )
    return(matched_sentences)

t4 = ['my name is daniel', 'your name is eric']

extract_matched_sentences(t4, t3)

{'daniel': [['my name is daniel']],
 'name': [['my name is daniel', 'your name is eric']]}

In [3]:
client = Client(n_workers=6, threads_per_worker=1, memory_limit='1GB')
client

Client Scheduler: tcp://127.0.0.1:57020 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 6 Cores: 6 Memory: 6.00 GB


In [4]:
files = str(here("./data/db/working/kaggle/id_model_inputs/01-04-found_terms.json.gzip"))
pth = os.path.join(files, "*.part")

In [5]:
paper_df = dd.read_json(pth, compression="gzip")

In [6]:
# about 6k rows
found_terms = (paper_df
                   .loc[paper_df["found_terms"] != {}]
                   .repartition(npartitions=paper_df.npartitions // 10)
              )

In [7]:
# found_terms["found_sent_idx"] = np.vectorize(extract_matched_sentences_i)(found_terms["sent_set"], found_terms["found_terms"])
found_terms["found_sent_idx"] = found_terms.map_partitions(lambda df: np.vectorize(extract_matched_sentences_i)(df["sent_set"], df["found_terms"]), meta="object")

In [8]:
# found_terms["found_sent"] = np.vectorize(extract_matched_sentences)(found_terms["text_sent_lower"], found_terms["found_sent_idx"])
found_terms["found_sent"] = found_terms.map_partitions(lambda df: np.vectorize(extract_matched_sentences)(df["text_sent_lower"], df["found_sent_idx"]), meta="object")

In [9]:
#found_terms.visualize()

In [9]:
start_time = datetime.now()

pth = str(here("./data/db/working/kaggle/id_model_inputs/02-found_sentences.json.gzip", warn=False))
found_terms.to_json(pth, compression="gzip") # about 3 minutes

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:02:27.939942


In [10]:
client.shutdown()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
